In [2]:
import cv2
import numpy as np

In [3]:
win_name = 'scan'
src = cv2.imread('./sample1.jpg') # 이미지 파일 읽기
img = src.copy()


In [31]:
resize = cv2.resize(img, dsize=(0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_AREA)
cv2.imshow("조절", resize)
cv2.waitKey(0) # param 시간동안 키보드 입력 대기 - 0 인경우 무한대기
cv2.destroyAllWindows()

In [5]:
gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 이미지를 흐리게 처리함 (noise 제거를 위해 사용)
edged = cv2.Canny(gray, 75, 250) # edged를 검출하는 함수 (img, minVal, maxVal)
cv2.imshow('gray', edged)  # 윈도우 창에 보여주기 (윈도우창 title, 이미지)
cv2.waitKey(0) # param 시간동안 키보드 입력 대기 - 0 인경우 무한대기
cv2.destroyAllWindows()

In [6]:
# 컨투어 찾기
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
len(contours)

3

In [7]:
draw = resize.copy()
cv2.drawContours(draw,  contours, -1, (0, 255, 0))
cv2.imshow("contours", draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

외곽선 관련 함수
- cv2.arcLength() : 외곽선의 길이를 반환
- cv2.contourArea() : 외곽선이 감싸는 영역의 면적을 반환
- cv2.boundingRect() : 주어진 점을 감싸는 최소 크기의 사각형(바운딩 박스)를 반환
- cv2.minEnclosingCircle() : 주어진 점을 감싸는 최소 크기의 원을 반환
- cv2.minAreaRect() : 주어진 점을 감싸는 최소 크기의 회전된 사각형을 반환
- cv2.approxPolyDP() : 외곽선을 근사화(단순화)
- cv2.fitEllipse() : 주어진 점에 적합한 타원을 반환
- cv2.fitLine() : 주어진 점에 적합한 직선을 반환
- cv2.isContourConvex() : 컨벡스인지를 검사
- cv2.convexHull() : 주어진 점으로부터 컨벤스 헐을 반환
- cv2.convexityDefects() : 주어진 점과 컨벡스 헐로부터 컨벡스 디펙트를 반환

참조 사이트
 - https://deep-learning-study.tistory.com/232
 

In [10]:
# 컨투어들 중에 영역 크기 순으로 정렬
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
for c in contours:
    peri = cv2.arcLength(c, True) # 외곽선 길이
    # print(peri)
    verticles = cv2.approxPolyDP(c, 0.02 * peri, True) # 외곽선 근사화
    if len(verticles) == 4 : 
        break
pts = verticles.reshape(4, 2) # 배열을 4 * 2 크기로 조정

In [19]:
print(pts)
print(pts.sum(axis=1))
print(np.diff(pts, axis=1))
print("좌상단", pts[np.argmin(pts.sum(axis=1))])

[[512  68]
 [ 50  73]
 [ 38 748]
 [552 736]]
[ 580  123  786 1288]
[[-444]
 [  23]
 [ 710]
 [ 184]]
좌상단 [50 73]


In [20]:
# 각각의 좌표 찾기
sumXY = pts.sum(axis=1)
diff = np.diff(pts, axis=1)

topLeft = pts[np.argmin(sumXY)]
bottomRight = pts[np.argmax(sumXY)]
topRight = pts[np.argmin(diff)]
bottomLeft = pts[np.argmax(diff)]

np.float32([topLeft, topRight, bottomRight, bottomLeft])

array([[ 50.,  73.],
       [512.,  68.],
       [552., 736.],
       [ 38., 748.]], dtype=float32)

In [23]:
# 사진을 변환할 때 사용할 서류의 높이
widthTop = abs(topRight[0] - topLeft[0])
widthBottom = abs(bottomRight[0] - bottomLeft[0])
heightRight = abs(topRight[1] - bottomRight[1])
heightLeft = abs(topLeft[1] - bottomLeft[1])
print(widthBottom, widthTop, heightLeft, heightRight)

width = max([widthTop, widthBottom])
height = max([heightRight, heightLeft])

514 462 675 668


In [25]:
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
pts2 = np.float32([[0,0], [width, 0], [width, height], [0, height]])

In [33]:
matrix = cv2.getPerspectiveTransform(pts1, pts2) # 좌표를 변환하기 위해 사용할 변환행렬
result = cv2.warpPerspective(resize, matrix, (width, height)) # 이미지 변환(변환행렬 적용)
cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()